In [ ]:
#| echo: false
#| hide: true

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

<!-- ```{shinylive-python}
#| standalone: true
#| viewerHeight: 1000
#| viewerWidth: 600

from shiny import App, ui, render
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

app_ui = ui.page_fluid(
    ui.input_slider("n", "Number of bins", min=0, max=100, value=40, step=1),
    ui.input_slider("noise", "Noise level", min=0, max=1, value=0.5, step=0.01),
    ui.input_slider("beta", "Slope of regression line", min=-2, max=2, value=0.5, step=0.1),
    ui.output_plot("plot")
)
def server(input, output, session):
    @output
    @render.plot(alt="Average Y by binned X", width=600, height=400)
    def plot():
        rng = np.random.default_rng(42)

        x = rng.normal(size=1000)
        y = input.beta() * x + input.noise() * rng.normal(size=1000)

        data = pd.DataFrame({'x': x, 'y': y})

        sns.scatterplot(x='x', y='y', data=data, alpha=0.3)
        sns.regplot(x='x', y='y', x_bins=input.n(), data=data, ci=None, fit_reg=False, marker="x", color='red')

        plt.xlabel('X-axis')
        plt.ylabel('Y-axis')
        plt.show()

app = App(app_ui, server)
``` -->

## Inference in Regression

Last lecture we introduced **linear regression** as a way to model the relationship between a response variable $Y$ and some predictor variable $X$. The model assumes that the relationship can be described by a linear equation:

$$ Y = \beta_0 + \beta_1 X + \epsilon $$

where $\beta_0$ is the intercept, $\beta_1$ is the slope, and $\epsilon$ is the error term. The error term captures the variability in $Y$ that cannot be explained by $X$, and we assume it follows a normal distribution with mean 0 (i.e. the deviations from the true regression line are zero on average) and some variance $\sigma^2$.

Throughout this course, we have focused on **sampling variability** and how it affects our ability to make inferences about a population based on a sample. In the context of regression, this means that if we take different samples from the same population, we will get different estimates of the regression coefficients $\beta_0$ and $\beta_1$. This variability is due to the random nature of sampling and the inherent noise in the data. 

The app below illustrates this concept by allowing you to draw samples from a population and see how the estimated regression coefficient ($\hat{\beta}_1$) varies across samples. Notice that the true slope of the regression line is fixed, but the estimated slope varies due to sampling variability. 

You can also see how the stability of the estimate improves as the sample size increases, as well as the confidence implied by the $p$-value of the hypothesis test for the slope ($H_0: \beta_1 = 0$). We will unpack this in more detail shortly.



:::{.column-screen-inset}
```{shinylive-python}
#| standalone: true
#| viewerHeight: 700

from shiny import App, ui, render, reactive
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

# Fixed population
rng = np.random.default_rng(123)
N_POP = 2000
x_pop = rng.normal(size=N_POP)
true_beta = 0.5
y_pop = true_beta * x_pop + rng.normal(scale=1, size=N_POP)
population = pd.DataFrame({"x": x_pop, "y": y_pop})

app_ui = ui.page_fluid(
    # Title
    ui.div(
        ui.h2("Sampling Variability in Regression Inference"),
        style="text-align:center; margin-bottom:20px; margin-top:20px; font-weight:bold;"
    ),
    # Top row with inputs
    ui.layout_columns(
        ui.input_slider("n", "Sample size", min=5, max=200, value=50, step=5, width="300px"),
        ui.input_numeric("perm", "Number of permutations", value=500, min=100, max=2000, step=100, width="200px"),
        ui.input_action_button("resample", "Draw New Sample", width="200px"),
    ),
    # Second row with two side-by-side plots
    ui.navset_tab(
        ui.nav_panel("Visualization",
            ui.div(
                ui.output_text("coef_text"),
                style="text-align:center; font-size:18px; margin-top:10px; margin-bottom:10px;"
            ),
            ui.layout_columns(
                ui.output_plot("scatter_plot"),
                ui.output_plot("perm_plot"),
            )
        ),
        ui.nav_panel("Permutation Table",
            ui.div(
                "First 50 rows of original sample and first 10 permutations:",
                style="text-align:center; font-size:16px; margin-top:10px; margin-bottom:10px;"
            ),
            ui.output_table("perm_table")
        )
    )
)

def server(input, output, session):
    # ✅ Store the current sample as state
    current_sample = reactive.value(pd.DataFrame())

    # Update sample when button is clicked or sample size changes
    @reactive.effect
    @reactive.event(input.resample, input.n)
    def _():
        idx = rng.choice(len(population), size=input.n(), replace=False)
        current_sample.set(population.iloc[idx])

    # ✅ Compute regression results based on stored sample
    @reactive.calc
    def regression_results():
        data = current_sample()
        if data.empty:
            return {"beta_hat": np.nan, "p_val": np.nan, "perm_coefs": np.array([]), "perm_table": pd.DataFrame()}

        model = smf.ols("y ~ x", data=data).fit()
        beta_hat = model.params["x"]
        p_val = model.pvalues["x"]

        perm_coefs = []
        perm_tables = []
        for i in range(input.perm()):
            y_perm = rng.permutation(data["y"])
            perm_model = smf.ols("y ~ x", data=data.assign(y=y_perm)).fit()
            perm_coefs.append(perm_model.params["x"])

            if i < 10:  # store first 10 permutations for table
                df_perm = pd.DataFrame({"x": data["x"].values, f"y_perm_{i+1}": y_perm})
                perm_tables.append(df_perm[[f"y_perm_{i+1}"]])

        # Merge the first few permutations into one table
        if perm_tables:
            perm_table_df = pd.concat([data[["x", "y"]].reset_index(drop=True)] + perm_tables, axis=1)
        else:
            perm_table_df = pd.DataFrame()
        
        # compute permutation p-value
        perm_coefs = np.array(perm_coefs)
        # two-tailed p-value
        p_val_perm = np.mean(np.abs(perm_coefs) >= np.abs(beta_hat))

        return {
            "beta_hat": beta_hat,
            "p_val": p_val,
            "p_val_perm": p_val_perm,
            "perm_coefs": np.array(perm_coefs),
            "perm_table": perm_table_df
        }

    @output
    @render.text
    def coef_text():
        res = regression_results()
        if np.isnan(res["beta_hat"]):
            return "Click 'Draw New Sample' to begin."
        return f"True slope: {true_beta:.3f}, Estimated slope: {res['beta_hat']:.3f}, Parametric p-value: {res['p_val']:.4f}, Permutation p-value: {res['p_val_perm']:.4f}"

    @output
    @render.plot
    def scatter_plot():
        data = current_sample()
        res = regression_results()

        plt.figure(figsize=(7, 5))
        sns.scatterplot(x="x", y="y", data=population, alpha=0.1, color="gray")

        if not data.empty:
            sns.scatterplot(x="x", y="y", data=data, color="blue", label="Sampled Points")
            x_vals = np.linspace(population["x"].min(), population["x"].max(), 100)
            y_vals = res["beta_hat"] * x_vals + data["y"].mean() - res["beta_hat"] * data["x"].mean()
            plt.plot(x_vals, y_vals, color="red", lw=2, label="Estimated Regression Line")
            # Add true population regression line
            plt.plot(x_vals, true_beta * x_vals, alpha=0.3, color="gray", linestyle="--", label="True Regression Line")
            plt.legend()


        plt.xlabel("X")
        plt.ylabel("Y")
        plt.xlim(population["x"].min(), population["x"].max())
        plt.ylim(population["y"].min(), population["y"].max())
        plt.title("Sampled Points and Fitted Regression Line")

    @output
    @render.plot
    def perm_plot():
        res = regression_results()
        if len(res["perm_coefs"]) == 0:
            plt.figure()
            plt.text(0.5, 0.5, "Click 'Draw New Sample' to start.", ha="center", va="center")
            return

        plt.figure(figsize=(7, 4))
        sns.histplot(res["perm_coefs"], bins=30, kde=False, color="gray")
        plt.axvline(res["beta_hat"], color="red", lw=2, label="Observed β̂")
        plt.axvline(true_beta, color="blue", lw=2, linestyle="--", label="True β")
        plt.xlabel(r"Estimated slope ($\hat{\beta}$)")
        plt.ylabel("Count")
        plt.xlim(min(-1, res["perm_coefs"].min()), max(1, res["perm_coefs"].max()))
        plt.title(r"Null Distribution of $\hat{\beta}$ (Permutation Test)")
        plt.legend()
    @output
    @render.table
    def perm_table():
        res = regression_results()
        df = res["perm_table"]
        if df.empty:
            return pd.DataFrame()
        return df.head(50)  # Only show first 50 rows

app = App(app_ui, server)
```
:::

:::{.callout-note title="Fitting the regression model" collapse="true"}
As in the previous lecture, we use the `statsmodels` library to fit the regression model. 

Under the hood, the app is doing the following:

```python
# Fixed population
rng = np.random.default_rng(123)
N_POP = 2000
x_pop = rng.normal(size=N_POP)
true_beta = 0.5
y_pop = true_beta * x_pop + rng.normal(scale=1, size=N_POP)
population = pd.DataFrame({"x": x_pop, "y": y_pop})
```
This creates a fixed population of 2000 points with a known slope of 0.5. The app then allows you to draw samples from this population and fit a regression model to the sampled data.

More specifically, we subsample the population to create a new sample of size `n`:

```python
idx = rng.choice(len(population), size=n, replace=False)
data = population.iloc[idx]
```
Then we fit the regression model using `statsmodels`:
```python
model = smf.ols('y ~ x', data=data).fit()
    beta_hat = model.params['x']
```
:::


### Hypothesis Testing in Regression

In regression analysis, we often want to test hypotheses about the relationship between the predictor variable $X$ and the response variable $Y$. The most common hypothesis to test is whether the slope of the regression line ($\beta_1$) is different from zero, which would indicate that there is a relationship between $X$ and $Y$.

So we can set up the following hypotheses:
$$
\begin{align*}
H_0:& ~\beta_1 = 0 \quad \text{(no relationship)} \\
H_1:& ~\beta_1 \neq 0 \quad \text{(there is a relationship)}
\end{align*}
$$

We're back in our comfort zone now. How can we simulate the distribution of the estimated slope $\hat{\beta}_1$ under the null hypothesis? We'll use a **permutation test** -- take a shot at implementing it in the code cell below.


::::{.panel-tabset}

## Exercise

```{pyodide}
#| exercise: hypothesis-testing-regression
#| caption: Write a permutation test to simulate the distribution of the estimated slope under the null hypothesis that there is no relationship between X and Y.
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np

n = 100
x = rng.normal(size=100)
y = 0.5 * x + 0.5 * rng.normal(size=100)
data = pd.DataFrame({'x': x, 'y': y})
rng = np.random.default_rng(42)

def permutation_test_regression(data, n_permutations=1000):
    # first, fit the regression model (on the original data)
    model = smf.ols('y ~ x', data=data).fit()
    beta_hat = model.params['x']
    
    # TODO: implement the permutation test. 
    # You should be able to return the observed slope, 
    # the array of permuted slopes, and the p-value 
    # associated with the test.

    return {
        "beta_hat": beta_hat,
        "perm_beta_hats": perm_beta_hats,
        "p_val": p_val,
    } 
```

## Hint
::: {.hint exercise="hypothesis-testing-regression"}
To implement the permutation test, you will need to:

1. Fit the regression model to the original data to get the observed slope $\hat{\beta}_1$.
2. For each permutation:
   - Permute the response variable $Y$ to break the relationship with $X$.
   - Fit the regression model to the permuted data.
   - Store the estimated slope $\hat{\beta}_1$ from the permuted data.
3. Compute the p-value as the proportion of permuted slopes that are greater than or equal to the observed slope in absolute value (two-tailed test).

:::

## Solution
::: {.solution exercise="hypothesis-testing-regression"}

```python
def permutation_test_regression(data, n_permutations=1000):
    # first, fit the regression model (on the original data)
    model = sm.ols('y ~ x', data=data).fit()
    beta_hat = model.params['x']
    
    perm_beta_hats = [] # list to store permuted slopes
    for _ in range(n_permutations):
        # permute the response variable y
        # this breaks the relationship between x and y
        y_perm = rng.permutation(data['y'])
        # fit the regression model to the permuted data
        perm_model = sm.OLS(y_perm, X).fit()
        # store the estimated slope
        perm_beta_hats.append(perm_model.params['x'])
    # the slopes from the permuted data are
    # your null distribution
    perm_beta_hats = np.array(perm_beta_hats)
    # compute p-value
    p_val = np.mean(np.abs(perm_beta_hats) >= np.abs(beta_hat))
    
    return {
        "beta_hat": beta_hat,
        "perm_beta_hats": perm_beta_hats,
        "p_val": p_val,
    }
```
:::
::::


This is precisely the permutation test that generates the null distribution displayed in the app. 

Notice that the permuation test $p$-value is nearly identical to the parametric $p$-value computed by `statsmodels` using the assumption that the errors are normally distributed. 

:::{.callout-note title="Parametric inference in regression" collapse="true"}
The app uses `statsmodels` to compute the regression coefficients and their associated p-values. To get these $p$-values (without doing simulations / randomization), you have to make assumptions about the underlying probability distributions. 

In the linear regression model, the typical assumption is that (as we discussed in the previous lecture) the errors $\epsilon$ are normally distributed. This makes the $Y$ values themselves normal random variables (i.e. $Y \sim \mathcal{N}(\beta_0 + \beta_1 X, \sigma^2)$). This assumption allows us to use the properties of the normal distribution to figure out analytical expressions for the uncertainty in the estimated coefficients (a.k.a. the standard errors) and the $p$-values for hypothesis tests.

The details of the parametric inference are beyond the scope of this course but you are encouraged to take more advanced courses in statistics or econometrics to learn more about it! 
:::


### Confidence Intervals for Regression Coefficients

$p$-values are nice, but they don't tell us the whole story. We might want to get a sense of the range of plausible values for the regression coefficients, because the effect size is important too.

:::{.callout-warning title="Effect Size" collapse="true"}
**Effect size** refers to the magnitude of a quantity you are interested in estimating -- often a difference in averages or a regression coefficient. 

For a simple example, let's go all the way back to coin flips. We saw examples where we tried to estimate the probability of seeing a certain set of outcomes if the coin was fair. We saw that with lots of coin flips, we were able to confidently detect a biased coin. But we saw an example where the coin was **very** biased (only 25% heads). What if the coin was just a little biased (say 51% heads)? This is a much smaller effect size, and it would be harder to detect with the same number of coin flips -- but with enough flips we could still detect it. Is a 1% difference in probability of heads worth fighting over? What about a 0.1% difference? 

This comes up a lot in scientific research. Say you are testing a new drug and you find that it reduces symptoms by 5% compared to a placebo. Is that a meaningful effect? What if the drug only reduces symptoms by 0.5%? Producing the drug might be very expensive, so a small effect size might not be worth the cost (or possibly the risk of side effects).

::: 

In addition to hypothesis testing, we can also use bootstrapping to construct confidence intervals for the regression coefficients. 

Here's some code that does this:

In [ ]:
# | code-fold: show

import statsmodels.formula.api as sm
import pandas as pd
import numpy as np


def bootstrap_regression_ci(data, x_col, y_col, n_bootstraps=1000, alpha=0.05):
    n = len(data)
    coefs = []
    for _ in range(n_bootstraps):
        sample = data.sample(n, replace=True)  # <1>
        model = sm.ols(f"{y_col} ~ {x_col}", data=sample).fit()  # <2>
        coefs.append(model.params[x_col])  # <3>
    coefs = np.array(coefs)
    lower_bound = np.percentile(coefs, 100 * alpha / 2)  # <4>
    upper_bound = np.percentile(coefs, 100 * (1 - alpha / 2))
    return coefs, lower_bound, upper_bound


rng = np.random.default_rng(123)
# Generate synthetic data
x = rng.normal(size=100)
y = 1.5 * x + 2 + rng.normal(size=100)
data = pd.DataFrame({"x": x, "y": y})

model = sm.ols("y ~ x", data=data).fit()  # <5>
print(model.summary())  # <6>

bootstrap_coefs, lower_bound, upper_bound = bootstrap_regression_ci(
    data, "x", "y", n_bootstraps=10000
)
print("#" * 40)
print(f"Bootstrap CI for slope: [{lower_bound:.3f}, {upper_bound:.3f}]")


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.699
Model:                            OLS   Adj. R-squared:                  0.696
Method:                 Least Squares   F-statistic:                     227.5
Date:                Sun, 27 Jul 2025   Prob (F-statistic):           2.71e-27
Time:                        18:50:46   Log-Likelihood:                -132.94
No. Observations:                 100   AIC:                             269.9
Df Residuals:                      98   BIC:                             275.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.0933      0.093     22.569      0.0

1. Bootstrap the data to create multiple samples (same size as the original sample, sampled with replacement).
2. Fit the regression model to each bootstrap sample.
3. Store the estimated coefficients from each bootstrap sample.
4. Compute the confidence intervals based on the distribution of the bootstrap estimates.
5. Fit the regression model to the original data to get the point estimates.
6. Print statsmodels summary table with the confidence intervals.

The summary table has lots of extra information you can ignore for now, but the key part is the `coef` column which contains the point estimates of the regression coefficients, and the `[0.025, 0.975]` columns which contain the lower and upper bounds of the 95% confidence intervals.

We also printed the bootstrapped confidence intervals, and you can see that they are very similar to the ones computed by `statsmodels` using the normality assumption.

Still confused? Try the following app to visualize what's going on:

```{shinylive-python}
#| standalone: true
#| viewerHeight: 500

from shiny import App, ui, render, reactive
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

# Original population
rng = np.random.default_rng(123)
N_POP = 500
x_pop = rng.normal(size=N_POP)
true_beta = 0.5
y_pop = true_beta * x_pop + rng.normal(scale=1, size=N_POP)
population = pd.DataFrame({"x": x_pop, "y": y_pop})

# Fit regression on full population
pop_model = smf.ols("y ~ x", data=population).fit()
pop_slope = pop_model.params["x"]

app_ui = ui.page_fluid(
    ui.layout_columns(
        ui.input_slider("n", "Sample Size", min=10, max=500, value=100),
        ui.input_action_button("resample", "Resample 1x"),
        ui.input_action_button("resample10", "Resample 10x"),
        ui.input_action_button("resample100", "Resample 100x"),
    ),
    ui.output_plot("scatter_plot"),
    ui.output_plot("hist_plot"),
)

def server(input, output, session):
    current_sample = reactive.value(pd.DataFrame())
    bootstrap_coefs = reactive.value([])

    def take_sample():
        sample = population.sample(n=input.n(), replace=True)
        model = smf.ols("y ~ x", data=sample).fit()
        bootstrap_coefs.set(bootstrap_coefs() + [model.params["x"]])
        current_sample.set(sample)

    @reactive.effect
    @reactive.event(input.resample)
    def resample_once():
        take_sample()

    @reactive.effect
    @reactive.event(input.resample10)
    def resample_ten():
        for _ in range(10):
            take_sample()

    @reactive.effect
    @reactive.event(input.resample100)
    def resample_hundred():
        for _ in range(100):
            take_sample()

    @output
    @render.plot
    def scatter_plot():
        plt.figure(figsize=(6, 4))
        # Plot population faintly
        sns.scatterplot(x="x", y="y", data=population, alpha=0.1, color="gray", label="Population")

        # Population regression line
        x_vals = np.linspace(population["x"].min(), population["x"].max(), 100)
        y_vals = pop_model.params["Intercept"] + pop_model.params["x"] * x_vals
        plt.plot(x_vals, y_vals, color="black", lw=2, linestyle="--", label="Population Line")

        # If we have a current sample, overlay it
        sample = current_sample()
        if not sample.empty:
            sns.scatterplot(x="x", y="y", data=sample, color="blue", alpha=0.6, label="Bootstrap Sample")
            model = smf.ols("y ~ x", data=sample).fit()
            y_sample = model.params["Intercept"] + model.params["x"] * x_vals
            plt.plot(x_vals, y_sample, color="red", lw=2, label="Sample Line")

        plt.xlabel("x")
        plt.ylabel("y")
        plt.title("Population (faint) and Current Bootstrap Sample")
        plt.legend()

    @output
    @render.plot
    def hist_plot():
        coefs = bootstrap_coefs()

        plt.figure(figsize=(6, 4))
        if coefs:
            sns.histplot(coefs, bins=20, kde=False, color="lightblue")

            # Vertical lines at quantiles
            q_low, q_high = np.quantile(coefs, [0.025, 0.975])
            plt.axvline(q_low, color="purple", linestyle="--", lw=2, label="2.5% Quantile")
            plt.axvline(q_high, color="purple", linestyle="--", lw=2, label="97.5% Quantile")

        # Vertical line at population slope
        plt.axvline(pop_slope, color="black", lw=2, label="Observed (Population) β̂")

        plt.xlabel("Bootstrap Slope Estimates")
        plt.ylabel("Frequency")
        plt.title("Bootstrap Distribution of β̂")
        plt.legend()

app = App(app_ui, server)
```